# Error_recovery task (with langchain and memory)

In [ ]:
import os, datetime
os.environ["OPENAI_API_KEY"] = open("key.txt").read().strip() # run this code before engaging LLM model
# import json
from typing import Optional, List, Dict, Any
# import os, json, re, datetime
# import traceback
# from langchain.schema import AIMessage, HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent, AgentExecutor # the tools needed to create a sort of chain with directives
from langchain.tools import StructuredTool,Tool
from langchain.memory import ConversationBufferMemory

def get_current_iso_8601_datetime_with_utc_offset(dummy: Optional[str] = None) -> str: # we pass it to the model as to correctly create datetime_range_lowerbound
    # returns now with local offset (simplified)
    # dummy: Optional[str] = None is needed perche senno' riceviamo un error, perche di default il modello aggiunge sempre un arg, anche se 
    # abbiamo Tool e non StructuredTools
    print('datetime hit')
    return f" current datetime: {datetime.datetime.now(datetime.timezone.utc).astimezone().isoformat()}"


def search_calendar_events(title: Optional[str]=None, attendee_names: Optional[List[str]]=None,
                           datetime_range_lowerbound: Optional[str]=None, datetime_range_upperbound: Optional[str]=None,
                           calendar_id: Optional[str]=None) -> List[Dict[str,Any]]: # all args can be null without throwing errors
    """
    Simulated search_calendar_events.
    - If lowerbound provided but not upperbound -> raise ValueError (the error mode from guidelines) so the model understand
      if it has to brodening args(create datetime_range_upperbound), or narrow it (omit even datetime_range_lowerbound) .
    - Applies simple filters; returns list of events (possibly empty).
    """

    _FAKE_EVENTS = [ # Imitate the events in google calender 
    {"calendar_event_id":"1","title":"1:1 Jake | Jill","attendee_names":["Jake","Jill"],"start_datetime":"2025-05-28T11:30:00-07:00","end_datetime":"2025-05-28T12:00:00-07:00"},
    {"calendar_event_id":"2","title":"Team Event","attendee_names":["Jake","Jill","Jeff"],"start_datetime":"2025-05-27T09:00:00-07:00","end_datetime":"2025-05-27T17:00:00-07:00"}
    ]
    if not attendee_names:
      if (datetime_range_lowerbound is not None) ^ (datetime_range_upperbound is not None):
         print('no datetime_range_lowerbound/upperbound hit')
        # raise ValueError("datetime_range_lowerbound and datetime_range_upperbound must be both provided or both omitted.")
         return "datetime_range_lowerbound and datetime_range_upperbound must be both provided or both omitted." # Helpful error like returns that can helps the model create right args
    # simple filter simulation
    # if not isinstance(attendee_names,list) or not all(isinstance(x,str) for x in attendee_names):
    #     return 'attendee_names has to be a list of strings' # Helpful error like returns that can helps the model create right args
    results = [] # result will be not empty if there is a data match in the filter
    for ev in _FAKE_EVENTS:
        if title and title.lower() not in ev["title"].lower(): # confronta il titolo dell arg con il titolo dei _FAKE_EVENTS
            continue
        if attendee_names:
            # require any overlap
            if not any(a for a in attendee_names if any(a.lower() in x.lower() for x in ev["attendee_names"])):
                continue
        # ignoring datetime filters for brevity
        results.append(ev)
    if not results:
        return 'No event found'  # Helpful error like returns that can helps the model create right args
    else:
       return results # result can be empty come quando gli args non sono completi, o possono contenere dati se l operazione va a buon termine


current_datetime_tool = Tool(  # Tool non da' l error se non passiamo args alla funzione. se passiamo arge allora usiamo StructuredTool
    func=get_current_iso_8601_datetime_with_utc_offset,
    name='get_current_iso_8601_datetime_with_utc_offset',
    description='Get the current iso_8601 datetime.No args needed'
)

calendar_event_tool = StructuredTool.from_function( # StructuredTool e usato quando abbiamo args nella funzione. 
    # from_function serve a non avere error dagli args. in questo modo diciamo al modello il type degli args cosi' come descritti nella funzione
    # (title: Optional[str]=None, attendee_names: Optional[List[str]]=None...ecc), cosi' il modello capisce quali args e args type generare, e non avremo 
    # 'errore di validazione Pydantic'
    func=search_calendar_events,
    name='search_calendar_events',
    # description='Search calendar events. Optional args: title:Optional[str]=None, attendee_names: Optional[List[str]]=None, datetime_range_lowerbound:Optional[str]=None, datetime_range_upperboundOptional[str]=None, calendar_id:Optional[str]=None).'
    description="Search calendar events with optional filters."
)

tools = [current_datetime_tool, calendar_event_tool]



system_message = f"""
You are an assistant that outputs valid arguments for the function search_calendar_events(title, attendee_names, datetime_range_lowerbound, datetime_range_upperbound, calendar_id).
  If a tool call fails or returns an error, do not panic:
    Rules:
      - Understand by context if you need to invoke a tool or if it is a general question.
      - Understand the user intends and call the right tools (e.g search_calendar_events).
      - If a datetime_range is requested, include both lowerbound and upperbound in ISO8601 format; otherwise omit both.
      - ALWAYS invoke get_current_iso_8601_datetime_with_utc_offset tool if no date lowerbound is mentioned in the user_input, and use the output data as a args for the calendar tool.
      - DO NOT **hallucinate** about date lowerbound. ALWAYS use the date lowerbound you get by invoking get_current_iso_8601_datetime_with_utc_offset tool.
      - If user mentions a person (e.g., 'with Jhon'), prefer attendee_names, skip the other args and DO NOT invoke date lowerbound tool.
      - Keep keys exactly: title, attendee_names, datetime_range_lowerbound, datetime_range_upperbound, calendar_id.
      - Decide whether you can retry by fixing or broadening parameters.
      - BEFORE calling any tool: produce a short HUMAN-FACING plan sentence prefixed with "assistant_plan:" describing what you will do next (example: "assistant_plan: i'll check the calender to se if i find some Events").
      - AFTER tool outputs, produce a clear HUMAN-FACING summary prefixed with "assistant_summary:" describing the result and next step (example: "assistant_summary: No events has been founded, i will retry with new parameters").
      - Do not include JSON or chain-of-thought. The only 'Thinking' you will output are 'assistant_plan' and 'assistant_summary'.
      """

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_message),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad") # riconosce gli error throwed dai tools e aggiusta gli args 
])

from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseMessage, HumanMessage


memory = ConversationBufferMemory( #ConversationBufferMemory( # istanza della classe SafeConversationMemory. Qui passiamo anche questi parametri definiti qua dentro alla classe
    memory_key="chat_history",
    return_messages=True,
    input_key="input"  # 👈 forza un solo input
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)

agent = create_openai_functions_agent( # creiamo il nostro agente strutturato
    llm=llm,
    tools = tools,
    prompt=prompt
)


agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools= tools, 
    memory=memory,
    verbose=True,
    handle_parsing_errors=True, 
)


C:\Users\ale\AppData\Local\Temp\ipykernel_26484\1326009508.py:149: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory( #ConversationBufferMemory( # istanza della classe SafeConversationMemory. Qui passiamo anche questi parametri definiti qua dentro alla classe


In [2]:
user_input = "What is my next one on one?" 
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"]) # output where you get the answers





> Entering new AgentExecutor chain...

Invoking: `get_current_iso_8601_datetime_with_utc_offset` with `current_datetime`
responded: assistant_plan: I'll check the calendar to see if I can find any upcoming one-on-one events. Since no specific date range is provided, I'll first get the current date and time.  


datetime hit
 current datetime: 2025-11-17T21:39:11.470356+01:00
Invoking: `search_calendar_events` with `{'title': 'one on one', 'datetime_range_lowerbound': '2025-11-17T21:39:11.470356+01:00', 'calendar_id': None}`


no datetime_range_lowerbound/upperbound hit
datetime_range_lowerbound and datetime_range_upperbound must be both provided or both omitted.
Invoking: `search_calendar_events` with `{'title': 'one on one', 'datetime_range_lowerbound': '2025-11-17T21:39:11.470356+01:00', 'datetime_range_upperbound': '2025-11-18T21:39:11.470356+01:00', 'calendar_id': None}`


No event foundassistant_summary: No upcoming one-on-one events were found in the calendar. If you have speci

In [3]:
user_input = "I should meet Jeff" #"Aggiungi una sessione di stretching alle 7:30."
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"]) # output where you get the answers





> Entering new AgentExecutor chain...

Invoking: `search_calendar_events` with `{'attendee_names': ['Jeff']}`
responded: assistant_plan: I'll search for calendar events that include Jeff as an attendee. 

Let's proceed with that.

[{'calendar_event_id': '2', 'title': 'Team Event', 'attendee_names': ['Jake', 'Jill', 'Jeff'], 'start_datetime': '2025-05-27T09:00:00-07:00', 'end_datetime': '2025-05-27T17:00:00-07:00'}]assistant_summary: I found an event titled "Team Event" that includes Jeff as an attendee. It is scheduled for May 27, 2025, from 9:00 AM to 5:00 PM (UTC-7). If you need more details or want to search for other events, just let me know!

> Finished chain.
assistant_summary: I found an event titled "Team Event" that includes Jeff as an attendee. It is scheduled for May 27, 2025, from 9:00 AM to 5:00 PM (UTC-7). If you need more details or want to search for other events, just let me know!


In [4]:
user_input = "Its fine, tnx alot" #"Aggiungi una sessione di stretching alle 7:30."
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"]) # output where you get the answers

# user_input = "hallo chat how are u?" #"Aggiungi una sessione di stretching alle 7:30."
# response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
# print(response["output"]) # output where you get the answers



> Entering new AgentExecutor chain...
assistant_summary: You're welcome! If you have any more questions or need assistance in the future, feel free to ask. Have a great day!

> Finished chain.
assistant_summary: You're welcome! If you have any more questions or need assistance in the future, feel free to ask. Have a great day!


# Error_recovery (no langchain or memory)

In [ ]:
# import os
# os.environ["OPENAI_API_KEY"] = open("key.txt").read().strip() # run this code before engaging LLM model

In [ ]:
import os, json, re, datetime
from typing import Optional, List, Dict, Any
from langchain_openai import ChatOpenAI

MODEL = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)  # o quello che usi

# ---------- Fake tools / datasource ---------
def get_current_iso_8601_datetime_with_utc_offset() -> str:
    # returns now with local offset (simplified)
    return datetime.datetime.now(datetime.timezone.utc).astimezone().isoformat()


# Simulated calendar DB
_FAKE_EVENTS = [
    {"calendar_event_id":"1","title":"1:1 Jake | Jill","attendee_names":["Jake","Jill"],"start_datetime":"2025-05-28T11:30:00-07:00","end_datetime":"2025-05-28T12:00:00-07:00"},
    {"calendar_event_id":"2","title":"Team Event","attendee_names":["Jake","Jill","Jeff"],"start_datetime":"2025-05-27T09:00:00-07:00","end_datetime":"2025-05-27T17:00:00-07:00"}
]


def search_calendar_events(title: Optional[str]=None, attendee_names: Optional[List[str]]=None,
                           datetime_range_lowerbound: Optional[str]=None, datetime_range_upperbound: Optional[str]=None,
                           calendar_id: Optional[str]=None) -> List[Dict[str,Any]]: # all args can be null without throwing errors
    """
    Simulated search_calendar_events.
    - If lowerbound provided but not upperbound -> raise ValueError (the error mode from guidelines).
    - Applies simple filters; returns list of events (possibly empty).
    """

    if (datetime_range_lowerbound is not None) ^ (datetime_range_upperbound is not None):
        raise ValueError("datetime_range_lowerbound and datetime_range_upperbound must be both provided or both omitted.") # This error will be passed 
    # to the model if it does not generate datetime_range_upperbound, the it ll create datetime_range_upperbound
    # simple filter simulation
    results = []
    for ev in _FAKE_EVENTS:
        if title and title.lower() not in ev["title"].lower(): # confronta il titolo dell arg con il titolo dei _FAKE_EVENTS
            continue
        if attendee_names:
            # require any overlap
            if not any(a for a in attendee_names if any(a.lower() in x.lower() for x in ev["attendee_names"])):
                continue
        # ignoring datetime filters for brevity
        results.append(ev)
    return results # result can be empty come quando gli args non sono completi, o possono contenere dati se l operazione va a buon termine



# ---------- Utilities ----------
def extract_json_from_model_text(text: str) -> str:
    """
    Remove markdown fences and return the JSON string.
    """
    # remove ```json / ``` markers
    text = re.sub(r"^```(?:json)?\n", "", text)
    text = re.sub(r"\n```$", "", text)
    return text.strip() # questo puo' essere utile per passare alle funzioni argomenti puliti, senza markdown fences 



def ask_model_for_args(user_request: str, # la funzione che addestra il modello a creare gli args per search_calendar_events tool
                       current_time: str, 
                       previous_error: Optional[str]=None,
                       attempt: int=1) -> Dict[str,Any]:
    
    """
    # User_request =  'What is my next one on one' or 'it should be with jill'
    # current_time = DateTime now()
    Grazie a questi soli dati, abbiamo addestrato il modello a creare args da essi, e addirittura args
    Nuovi 'smart' args (come datetime_range_upperbound), ed a imparare a migliorare eventualmente questi args
    in base a eventuali errors (definiti nello 'human_prompt') che si creano durante il percorso 

    Invia al modello un prompt per generare JSON arguments per search_calendar_events.
    Il modello deve RISPONDERE SOLO CON JSON.
    """

    system_prompt = """ 
        You are an assistant that outputs valid JSON arguments for the function search_calendar_events(title, attendee_names, datetime_range_lowerbound, datetime_range_upperbound, calendar_id).
        Rules:
         - Respond ONLY with a JSON object (no extra commentary).
         - If a datetime_range is requested, include both lowerbound and upperbound in ISO8601 format; otherwise omit both.
         - If user mentions a person (e.g., 'with Jill'), prefer attendee_names.
         - Keep keys exactly: title, attendee_names, datetime_range_lowerbound, datetime_range_upperbound, calendar_id.
    """

    human_prompt = f"""
        User request: "{user_request}"
        Current datetime: {current_time}
        Previous tool error (if any): {previous_error}
        Generate valid JSON arguments for search_calendar_events().
        Attempt: {attempt}
        Respond ONLY with a JSON object. (Allowed keys: title, attendee_names, datetime_range_lowerbound, datetime_range_upperbound, calendar_id)
        """
    
    messages = [
        {"role":"system","content":system_prompt},
        {"role":"user","content":human_prompt}
         ]
    
    resp = MODEL.invoke(messages) # il modello prende vita con le direttive system/ser
    raw = resp.content # il modello genera una risposta che genera gli args da passare alla funzione search_calendar_events()
    json_text = extract_json_from_model_text(raw) # puliamo gli args in modo da poter essere passati nella
    # function search_calendar_events() cosi che dentro essa possano essere filtrati correttamente

    try: # non ho capito bene cosa fa' questo try catch
        parsed = json.loads(json_text)
    except Exception as e: # Model didnt create correctly json args
        # If model returned non-JSON, try to salvage by searching for {...}
        m = re.search(r"\{.*\}", json_text, re.S)
        if m:
            parsed = json.loads(m.group(0))
        else:
            raise ValueError(f"Could not parse JSON from model output. Raw output:\n{raw}") from e
    print('parsed here =', parsed)
    return parsed
    # parsed data = {'title': 'one on one', 'attendee_names': [], 'datetime_range_lowerbound': '2025-10-29T17:42:24+01:00', 'datetime_range_upperbound': '2025-11-05T17:42:24+01:00', 'calendar_id': ''}



# ---------- Orchestration: error recovery loop ----------
def find_next_one_on_one(user_request: str, max_attempts: int = 3):
    current_time = get_current_iso_8601_datetime_with_utc_offset()
    previous_error = None # questo sara': 'datetime_range_lowerbound and datetime_range_upperbound must be both provided or both omitted'. se il modello non ha creato lower/upper bound
    # oppure "no_results" se una volta creando i bound non trova comunque nessun riscontro nel calendar tool
    attempt = 1

    while attempt <= max_attempts:
        print(f"\n--- Attempt {attempt} ---")
        # Ask model to produce the args
        try:
            args = ask_model_for_args(user_request, current_time, previous_error, attempt)
        except Exception as e: # se il modello non ha creato correttamente json data
            print("❌ Error parsing model output:", e)
            return # il loop si ferma perche c e il return, e in questo caso dovremmo aggiustare la funzione
          # che pulisce gli args creati dal modello

        # show assistant_plan (human-facing)
        print(f"assistant_plan: I'll call search_calendar_events with {args}")

        # call the tool and handle errors
        try: # args sono gli args che il modello ha creato per noi nella function ask_model_for_args()
        # args = {'title': 'one on one', 'attendee_names': [], 'datetime_range_lowerbound': '2025-10-29T17:42:24+01:00', 'datetime_range_upperbound': '2025-11-05T17:42:24+01:00', 'calendar_id': ''}
            result = search_calendar_events(
                title=args.get("title"),
                attendee_names=args.get("attendee_names"), # can be empty in the first round
                datetime_range_lowerbound=args.get("datetime_range_lowerbound"),
                datetime_range_upperbound=args.get("datetime_range_upperbound"), # can be empty in the first round
                calendar_id=args.get("calendar_id")
            )
        except Exception as e: # qua arriviamo quando il modello non passa tutti gli args datetime_range_lowerbound e datetime_range_upperbound al search_calendar_events tool, 
            # e quindi la function 'lancia' l error che viene catched qui (as 'e')
            previous_error = str(e) # str(e) = 'datetime_range_lowerbound and datetime_range_upperbound must be both provided or both omitted.'
            # si ripete il while loop e questo error viene passato al modello quando invochiamo la funzione ask_model_for_args(args,previus_error, args)
            # se otteniamo questo error, lo passiamo al modello e questi imparera da esso.
            print(f"❌ Tool error: {previous_error}")
            # Assistant will retry (broadening or removing args) — let the loop continue
            # but we do not nudge model yet; we let it retry by receiving previous_error in prompt
            attempt += 1
            continue # il loop continua perche non ce un return

        # If tool returned empty -> treat as "no results" and allow retry/broadening
        if not result:
            print("assistant_summary: search returned no results.")
            previous_error = "no_results"
            attempt += 1
            continue # il loop continua perche non ce un return

        # success!
        print("assistant_summary: Found events. Presenting results to user:")
        for ev in result:
            start = ev.get("start_datetime")
            title = ev.get("title")
            attendees = ", ".join(ev.get("attendee_names", []))
            print(f"- {title} | {start} | Attendees: {attendees}")
        return result # here stops the loop perche c e il return

    # if we exit loop without success (when the while loop is done)
    print("\nassistant_summary: I couldn't find matching events after several attempts. "
          "You can help me by providing more details (e.g., attendee name, exact title, or date range).")



    

In [3]:
# ---------- Demo scenario: matches the example ----------
if __name__ == "__main__":
    # 1) User asks broad question
    print("\nUser: What is my next one on one?")
    # Agent will try: first attempt the model might produce a lowerbound only -> simulated error
    res = find_next_one_on_one("What is my next one on one?") # in questo turn avviene un loop dove diciamo al modello quali args creare, di imparare dagli errors 
    # per creare eventualmente args migliori. ad esempio al primo turn forse il modello crea solo datetime_range_lowerbound e non datetime_range_upperbound. 
    # Il modello riceve l error, e in base a questi errors impara(fino al massimo di 3 volte dato il while loop), stabilisce se creare un effettivo datetime_range_upperbound args, o se toglierlo(come indicato nelle direttive del modello) 
    # in questo primo try il modello ha creato intelligentemente 'datetime_range_upperbound': '2025-10-30T17:42:24.873438+01:00', cio il giorno dopo alla stessa ora il 'datetime_range_lowerbound': '2025-10-29T17:42:24.873438+01:00'
    # se non lo avesse creato, l error, lo avrebbe addestrato a crearlo, oppure a non inserrire nessuno dei 2 bound
    # Quando non c'e stato nessun riscontro nel calender, allora il modello ha capito dall' error 'no result found' e a deciso di cambiare il 'datetime_range_upperbound' con un altra data: 'datetime_range_upperbound': '2025-11-05T17:42:24.873438+01:00' non ha 


User: What is my next one on one?

--- Attempt 1 ---
parsed here = {'title': 'one on one', 'attendee_names': [], 'datetime_range_lowerbound': '2025-11-18T16:41:42+01:00', 'datetime_range_upperbound': '2025-11-19T16:41:42+01:00', 'calendar_id': ''}
assistant_plan: I'll call search_calendar_events with {'title': 'one on one', 'attendee_names': [], 'datetime_range_lowerbound': '2025-11-18T16:41:42+01:00', 'datetime_range_upperbound': '2025-11-19T16:41:42+01:00', 'calendar_id': ''}
assistant_summary: search returned no results.

--- Attempt 2 ---
parsed here = {'title': 'one on one', 'attendee_names': [], 'datetime_range_lowerbound': '2025-11-18T16:41:42.086309+01:00', 'datetime_range_upperbound': '2025-12-18T16:41:42.086309+01:00', 'calendar_id': ''}
assistant_plan: I'll call search_calendar_events with {'title': 'one on one', 'attendee_names': [], 'datetime_range_lowerbound': '2025-11-18T16:41:42.086309+01:00', 'datetime_range_upperbound': '2025-12-18T16:41:42.086309+01:00', 'calendar_i

In [4]:
# 2) In example the assistant failed to find; user clarifies:
print("\nUser (follow-up): Yeah it might be named differently, and I remembered it should be with Jill")
res = find_next_one_on_one("It should be with Jill")  # pass hint to the same function
# In questo turn specifichiamo al modello che se nello user_input c'e' un nome, allora faremo il filtraggio in base ad esso, ritornandoci qualche dato
# Come notiamo, il modello non crea neanche 'datetime_range_lowerbound' nonostante gli abbiamo dato questo dato, e questo perche
# questa direttiva del systemMessage (" - If user mentions a person (e.g., 'with Jill'), prefer attendee_names.\n"),
#  influenza fortemente il modello, prendendo il sopravvento sulle altre direttive.


User (follow-up): Yeah it might be named differently, and I remembered it should be with Jill

--- Attempt 1 ---
parsed here = {'title': '', 'attendee_names': ['Jill'], 'datetime_range_lowerbound': '', 'datetime_range_upperbound': '', 'calendar_id': ''}
assistant_plan: I'll call search_calendar_events with {'title': '', 'attendee_names': ['Jill'], 'datetime_range_lowerbound': '', 'datetime_range_upperbound': '', 'calendar_id': ''}
assistant_summary: Found events. Presenting results to user:
- 1:1 Jake | Jill | 2025-05-28T11:30:00-07:00 | Attendees: Jake, Jill
- Team Event | 2025-05-27T09:00:00-07:00 | Attendees: Jake, Jill, Jeff
